# Final Alpha-Zero main code

# State

In [13]:
import numpy as np

class State:
    '''
    input: board_size, pieces, enemy_pieces
    한 state에 대한 정보와 게임 플레이 관련 메서드를 포함한 클래스이다.
    '''
    def __init__(self, board_size=3, pieces=None, enemy_pieces=None):
        self.board_size = board_size # 게임 보드 한 변의 길이

        # 자신의 수와 상대의 돌을 원핫으로 표현 (dim: (9, ))
        self.pieces = np.zeros(board_size * board_size, dtype=int) if pieces is None else np.array(pieces)
        self.enemy_pieces = np.zeros(board_size * board_size, dtype=int) if enemy_pieces is None else np.array(enemy_pieces)


    def piece_count(self, pieces):
        '''
        이 state의 전체 돌의 개수를 반환한다.
        '''
        return np.sum(pieces)

    def is_lose(self):
        '''
        이 state의 lose 여부를 반환한다.
        Note: 이 state에서 행동을 하기 전에 lose 여부를 확인한다.
        따라서 이 state가 lose라면, 이전 state에서 상대의 행동으로 상대가 승리한 경우이다.
        '''
        board = self.enemy_pieces.reshape(self.board_size, self.board_size)

        return any(
            np.all(line == 1) for line in
            np.vstack([
                board,  # 행
                board.T,  # 열
                np.diag(board),  # 대각선
                np.diag(np.fliplr(board))  # 반대 대각선
            ])
        )

    def is_draw(self):
        '''
        이 state의 draw 여부를 반환한다.
        '''
        return self.piece_count(self.pieces) + self.piece_count(self.enemy_pieces) == self.board_size * self.board_size

    def is_done(self):
        '''
        이 state의 게임 종료 여부를 반환한다.
        종료 조건: 상대방이 승리 or 무승부
        '''
        return self.is_lose() or self.is_draw()

    def next(self, action):
        '''
        이 state에서 주어진 action(칸)에 돌을 놓은 후 다음 state를 반환한다.
        '''
        pieces = self.pieces.copy()
        pieces[action] = 1
        return State(self.board_size, self.enemy_pieces, pieces)

    def legal_actions(self):
        '''
        가능한 행동(빈 칸의 위치)의 인덱스를 1차원 Numpy array로 반환한다.
        '''
        return np.where((self.pieces + self.enemy_pieces) == 0)[0]

    def to_feature(self):
        '''
        이 state를 신경망 입력 형태로 변환한다. (2, board_size, board_size)
        '''
        return np.stack([
            self.pieces.reshape(self.board_size, self.board_size),
            self.enemy_pieces.reshape(self.board_size, self.board_size)
        ], axis=0)

    def __str__(self):
        '''
        이 state를 문자열로 출력한다.
        '''
        ox = ('o', 'x') if np.sum(self.pieces) == np.sum(self.enemy_pieces) else ('x', 'o')
        board = np.full(self.board_size * self.board_size, '-')
        board[self.pieces == 1] = ox[0]
        board[self.enemy_pieces == 1] = ox[1]
        return '\n'.join([
            ''.join(board[i:i + self.board_size]) for i in range(0, self.board_size * self.board_size, self.board_size)
        ])

    def get_total_state(self):
        '''
        전체 state를 반환한다. (dim: (3, 3))
        1: 자신의 돌, -1: 상대의 돌
        '''
        board = np.zeros(self.board_size * self.board_size, dtype=int)
        board[self.pieces == 1] = 1   # 자신의 돌
        board[self.enemy_pieces == 1] = -1  # 상대의 돌 (반대 방향)
        return board.reshape(self.board_size, self.board_size)

# Dual Network

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    '''
    잔차 연결을 통해 학습을 돕는 Residual Block 클래스이다.
    '''
    def __init__(self, n_filters=128):
        super().__init__()
        self.conv1 = nn.Conv2d(n_filters, n_filters, kernel_size=3, padding=1, bias=False) # 첫번째 3X3 컨볼루션
        self.bn1 = nn.BatchNorm2d(n_filters) # 첫번째 배치 정규화
        self.conv2 = nn.Conv2d(n_filters, n_filters, kernel_size=3, padding=1, bias=False) # 두번째 3X3 컨볼루션
        self.bn2 = nn.BatchNorm2d(n_filters) # 두번째 배치 정규화

    def forward(self, x):
        shortcut = x # 잔차 연결용 - 원래 입력값을 보존
        x = F.relu(self.bn1(self.conv1(x))) # 첫번째 컨볼루션 -> 배치 정규화 -> Relu 활성화 함수
        x = self.bn2(self.conv2(x)) # 두번째 컨볼루션 -> 배치 정규화
        '''
        x(변형된 값)에 shortcut(원래 입력값)을 더한 후 Relu 활성화 함수를 적용하여 최종 출력을 반환한다.
        이를 통해 네트워크가 학습하기 쉬운 identity mapping(항등 함수)를 구현할 수 있다.
        '''
        return F.relu(x + shortcut)


class DualNetwork(nn.Module):
    '''
    AlphaZero에서 사용되는 정책과 가치 예측을 위한 듀얼 네트워크 구조를 구현한 것이다.
    '''
    def __init__(self, board_size, input_channels=2, n_filters=128, n_res_blocks=16):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, n_filters, kernel_size=3, padding=1, bias=False) # 초기 3X3 컨볼루션
        self.bn1 = nn.BatchNorm2d(n_filters) # 초기 배치 정규화
        '''
        RResidualBlock들을 연속적으로 쌓은 레이어 - 네트워크의 깊이를 증가시켜 복잡한 특징을 학습한다.
        '''
        self.res_blocks = nn.Sequential(
            *[ResidualBlock(n_filters) for _ in range(n_res_blocks)]
        )
        self.global_pool = nn.AdaptiveAvgPool2d(1) # 전역 평균 풀링 - 공간 차원 축소

        # policy(정책): 각 보드 위치에 대한 확률을 예측
        self.policy_head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(n_filters, board_size * board_size)
        )
        # value(가치): 현재 상태 가치를 -1과 1 사이의 값으로 예측
        self.value_head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(n_filters, 1),
            nn.Tanh()
        )

    def forward(self, x):
        '''
        input x : (배치 크기, 채널, 보드 크기, 보드 크기)
        '''
        x = F.relu(self.bn1(self.conv1(x))) # 초기 컨볼루션 -> 배치 정규화 -> Relu 활성화 함수
        x = self.res_blocks(x) # 여러 ResidualBlock들을 통과
        x = self.global_pool(x)
        # 정책(head)과 가치(head)를 각각 계산하여 반환
        policy = self.policy_head(x)
        value = self.value_head(x)
        return policy, value

# MCTS

In [15]:
C_PCUT = 1.0 # 탐험과 탐욕적 행동 선택 균형
PV_EVALUATE_COUNT = 100

In [16]:
from math import sqrt
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 추론
def predict(model, state, board_size):
    '''
    주어진 게임 상태(state)에 대해 신경망(model)을 사용하여
    정책(policy)과 가치(value)를 예측한다.
    '''
    # 입력 데이터 정규화
    x = np.array([state.pieces, state.enemy_pieces], dtype=np.float32) / 1.0 
    x = x.reshape(2, board_size, board_size).transpose(0, 1, 2).reshape(1, 2, board_size, board_size)
    x = torch.tensor(x, dtype=torch.float32).to(device)

    with torch.no_grad():
        policy, value = model(x) # 신경망으로 정책 및 가치 예측
        policy = policy.cpu().numpy().flatten()
        value = value.cpu().numpy().flatten()

    legal_actions = state.legal_actions()
    policies = policy[legal_actions]
    policy_sum = policies.sum()
    if policy_sum > 0:
        policies /= policy_sum
    else:
        policies = np.ones_like(policies) / len(policies)
    return policies, value[0]

class MCTSNode:
    '''
    MCTS에서 사용되는 각 노드를 구현한 클래스.
    각 노드는 하나의 게임 상태를 나타내며
    해당 상태에서 가능한 행동에 대한 정보를 담고 있다.
    '''
    def __init__(self, state, p, c_puct=C_PCUT):
        self.state = state
        self.p = p # 신경망에서 예측한 정책 확률
        self.w = 0 # 노드 누적 보상
        self.n = 0 # 노드 방문 횟수
        self.c_puct = c_puct
        self.child_nodes = None

    def evaluate(self, model):
        '''
        현재 노드를 평가하는 함수.
        종료된 게임 상태인지 확인하고
        종료되지 않았다면 자식 노드를 확장하거나, 탐색을 계속 진행하면서 가치를 갱신한다.
        '''
        # 1
        if self.state.is_done(): # 종료 상태인지 확인
            value = -1 if self.state.is_lose() else 0 # 패배: -1, 무승부: 0
            # 보상 누계와 시행 횟수 갱신
            self.w += value
            self.n += 1
            return value

        # 2
        # 자식 노드가 존재하지 않는 경우
        if not self.child_nodes:
            # 신경망으로 정책 및 가치 예측
            policies, value = predict(model, self.state, self.state.board_size)
            # 보상 누계와 시행 횟수 갱신
            self.w += value
            self.n += 1
            # 자식 노드 전개(확장)
            self.child_nodes = []
            for action, policy in zip(self.state.legal_actions(), policies):
              self.child_nodes.append(MCTSNode(self.state.next(action), policy, c_puct = self.c_puct))
            return value


        # 3
        else:
            # UCB1이 가장 큰 자식 노드를 평가해 가치 얻기
            # 탐색-활용(Exploration-Exploitation) 균형 전략 중 하나
            value = -self.next_child_node().evaluate(model)
            # 보상 누계와 시행 횟수 갱신
            self.w += value
            self.n += 1
            return value

    def next_child_node(self):
        '''
        UCB1(Upper Confidence Bound1) 공식을 사용하여
        탐험과 탐욕적 행동 간의 균형을 맞추는 방식으로 자식 노드를 선택한다.
        '''
        # 공식
        t = sum(child.n for child in self.child_nodes)
        pucb_values = []
        for child_node in self.child_nodes:
          pucb_values.append((-child_node.w/child_node.n if child_node.n else 0.0)+
                             self.c_puct * child_node.p * sqrt(t) / (1 + child_node.n))
        return self.child_nodes[np.argmax(pucb_values)] # 최대값 출력


def nodes_to_scores(nodes):
    '''
    노드 리스트를 시행 횟수 리스트로 변환한다.
    '''
    return [node.n for node in nodes]


def boltzman(xs, temperature):
    '''
    볼츠만 분포 계산
    '''
    xs = [x ** (1 / temperature) for x in xs]
    return [x / sum(xs) for x in xs]

# 몬테카를로 트리 탐색
def pv_mcts_scores(model, state, temperature):
    '''
    트리 탐색을 통해서 상태(state)에서 가능한 각 행동에 대한 확률 분포를 계산한다.
    여러 번의 시뮬레이션 후, 각 자식 노드의 방문 횟수를 점수로 변환하고,
    볼츠만 분포를 사용하여 확률을 결정한다.
    '''
    # 현재 상태의 루트 노드 생성
    root_node = MCTSNode(state, 0)

    # 시뮬레이션 반복 실행
    for _ in range(PV_EVALUATE_COUNT):
        root_node.evaluate(model)

    # 자식 노드 방문 횟수를 점수로 변환
    scores = nodes_to_scores(root_node.child_nodes)

    if temperature == 0: # 최대값인 경우에만 1
        action = np.argmax(scores)
        policy = np.zeros(len(scores))
        policy[action] = 1
    else:
        policy = boltzman(scores, temperature) # 볼츠만 분포를 기반으로 분산 추가
    return policy


# 몬테카를로 트리 탐색을 활용한 행동 선택
def pv_mcts_action(model, temperature=0):
    '''
    pv_mcts_scores()를 사용하여 상태에 대한 확률적 action selector를 반환하는 함수.
    temperature 값을 통해 탐험과 탐욕적 행동의 균형을 조절한다.
    '''
    def action_selector(state):
        scores = pv_mcts_scores(model, state, temperature) # 각 행동에 대한(상태에 기반) 확률분포 계산
        return np.random.choice(state.legal_actions(), p=scores) # 하나의 행동을 확률적으로 선택
    return action_selector

def mcts_simulation(root_node, model, simulations=100):
    '''
    여러 번의 시뮬레이션을 통해 주어진 루트 노드에서 가능한 행동들에 대한 확률을 계산한다.
    각 자식 노드의 방문 횟수를 기준을 정책을 계산한다.
    '''
    for _ in range(simulations):
        root_node.evaluate(model)
    # 방문 횟수 기준 정책 계산
    child_visits = [child.n for child in root_node.child_nodes]
    policy = np.array(child_visits) / sum(child_visits)
    return root_node, policy # 현재 게임 상태, 정책

# self play

In [17]:
def self_play_game(model, board_size=3, simulations=100, temperature=0):
    '''
    MCTS를 활용해 self-play 게임을 진행하고,
    각 상태에서 정책과 보상을 저장하여 상태, 정책, 보상 데이터를 반환한다.
    '''
    state = State(board_size=board_size)
    game_data = []

    while not state.is_done():
        # 현재 상태에서 MCTS 기반 점수 계산
        scores = pv_mcts_scores(model, state, temperature)

        # 점수를 고정 크기 정책으로 변환
        '''
        정책 벡터가 보드의 크기와 맞춰지고, 유효한 행동만 확률로 유지
        '''
        full_policy = np.zeros(board_size * board_size, dtype=np.float32)
        legal_actions = state.legal_actions()  # 유효한 행동 리스트
        full_policy[legal_actions] = scores
        game_data.append((state.to_feature(), full_policy))

        # 확률적으로 행동 선택
        action = np.random.choice(state.legal_actions(), p=scores)
        state = state.next(action)

    # 게임 종료 후 보상 계산
    reward = 1 if not state.is_lose() else -1
    game_data = [(s, p, reward) for s, p in game_data]
    return game_data

# Train Model

In [18]:
def train_model(model, scheduler, optimizer, game_data, epochs=1, batch_size=64):
    '''
    self-play로 얻은 데이터를 활용해 정책 및 가치함수를 학습한다.
    배치단위로 손실을 계산하고 최적화하며, 스케줄러를 통해 학습률을 조정한다.
    '''
    model.train()
    loss_fn_policy = torch.nn.CrossEntropyLoss()
    loss_fn_value = torch.nn.MSELoss()

    # 데이터를 PyTorch 텐서로 변환
    states, policies, values = zip(*game_data)

    states = torch.tensor(states, dtype=torch.float32).to(device)
    policies = torch.tensor(policies, dtype=torch.float32).to(device)
    values = torch.tensor(values, dtype=torch.float32).unsqueeze(1).to(device)

    dataset = torch.utils.data.TensorDataset(states, policies, values)
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        total_loss, total_policy_loss, total_value_loss = 0, 0, 0

        for inputs, targets_policy, targets_value in data_loader:
            inputs = inputs.to(device)
            targets_policy = targets_policy.to(device)
            targets_value = targets_value.to(device)

            # 모델예측
            predicted_policies, predicted_values = model(inputs)

            loss_policy = loss_fn_policy(predicted_policies, targets_policy)
            loss_value = loss_fn_value(predicted_values, targets_value)
            loss = 0.5*loss_policy + loss_value # policy가 너무 크게 나와서 앞에 0.5 곱해줌 이래도 되나..

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_policy_loss += loss_policy.item()
            total_value_loss += loss_value.item()

        # 에포크별 평균 기록
        training_metrics["loss"].append(total_loss / len(data_loader))
        training_metrics["policy_loss"].append(total_policy_loss / len(data_loader))
        training_metrics["value_loss"].append(total_value_loss / len(data_loader))

        scheduler.step() # scheduler는 마지막에

# Evaluate model

In [19]:
def evaluate_models(new_model, old_model, games=20, board_size=3):
    '''
    AlphaZero의 신경망 모델을 서로 대결시켜
    새 모델과 이전 모델의 성능을 비교하는 방식으로 평가한다.
    각 모델의 승리 횟수를 통해 모델의 우수성을 판단한다.

    new_model: 새로 학습된 신경망 모델
    old_model: 이전에 학습된 신경망 모델
    games: 두 모델이 대결한 게임의 수
    board_size: 게임판 크기(3x3)
    '''
    new_wins, old_wins = 0, 0 # 각 모델의 승리 횟수를 기록하는 변수
    temperature = 1.0

    for _ in range(games):
        state = State(board_size=board_size)
        current_player = new_model

        while not state.is_done():
            scores = pv_mcts_scores(current_player, state, temperature)
            action = np.random.choice(state.legal_actions(), p=scores)
            state = state.next(action)

            # 플레이어 교체
            current_player = old_model if current_player == new_model else new_model

        if state.is_lose():
            if current_player == new_model:
                old_wins += 1
            else:
                new_wins += 1

    # 승리 결과 기록
    evaluation_metrics["new_model_wins"].append(new_wins)
    evaluation_metrics["old_model_wins"].append(old_wins)
    return new_wins, old_wins

# w/ Robot

In [7]:
import sys
import os
# 현재 파일(main.ipynb)이 있는 디렉토리의 부모 디렉토리를 sys.path에 추가
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from Robotics.connect_CNC import *
from CV.square_board_recognition import *

In [21]:
board_size=3

def make_state(raw_state, alphazero_first):
    '''
    게임보드 인식으로 얻은 상태를 변환해 State 객체로 반환
    '''
    alphazero_pieces = 0 if alphazero_first else 1
    human_pieces = 1 if alphazero_first else 0

    my_actions = list(np.where(raw_state[0] == human_pieces)[0])
    enemy_actions = list(np.where(raw_state[0] == alphazero_pieces)[0])
    my_state = np.zeros((board_size*board_size, ))
    my_state[my_actions] = 1
    # print(my_state)
    enemy_state = np.zeros((board_size*board_size, ))
    enemy_state[enemy_actions] = 1
    # print(enemy_state)

    state = State(pieces = enemy_state, enemy_pieces = my_state)

    return state


In [22]:
device = torch.device("mps")

model = DualNetwork(board_size)
path = os.path.abspath(os.path.join(os.getcwd(), ".."))
model.load_state_dict(torch.load(f"{path}/model/trained_tic_tac_toe_model.pth", map_location=torch.device("cpu")))
model.to(device)  # 모델을 GPU로 이동
model.eval()

DualNetwork(
  (conv1): Conv2d(2, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (res_blocks): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=Tr

In [ ]:
# 커널 초기화 시 (시리얼 포트 최초 연결 시) 한 번 실행
serial = Serial('/dev/cu.usbserial-10') # write serial port name
serial.waiting_robot()

In [13]:
# 게임 정보
board_size=3
temperature = 0 # eval mode

# Alpha-Zero의 순서 지정
alphazero_first = input("Is Alpha-Zero first?: (True/False)")
agent_turn = "O" if alphazero_first else "X" # 선: "O"그리기 / 후: "X" 그리기
alphazero_turn = 1 if alphazero_first else 0

# Serial port 연결
serial = Serial('/dev/cu.usbserial-10')# write serial port name

state = State(board_size=board_size)

n_steps = 0
is_done = False

# 로봇: 게임보드 그리기
serial.draw_game_board()

# 게임 시작
while not is_done:
    n_steps += 1

    # Alpha-Zero turn
    if n_steps % 2 == alphazero_turn:
        print(f"n_steps: {n_steps}, alphazero_turn")
        scores = pv_mcts_scores(model, state, temperature)
        action = np.random.choice(state.legal_actions(), p=scores)

        # 로봇: Alpha-Zero의 행동 실행
        serial.send_to_robot(f"{agent_turn}{action}")

        state = state.next(action)
        str(state)
        is_done = state.is_done()

    else:
        print(f"n_steps: {n_steps}, human_turn")

        # 상대(사람) 행동 후 Camera start
        start_camera = input("Start Camera?(True/False)")

        if start_camera:
            # state 반환
            raw_state = return_state()

        # state 객체 꼴로 변환
        state = make_state(raw_state, alphazero_first)
        str(state)
        is_done = state.is_done()

    print(f"is_done: {is_done}")

if state.is_draw():
    print("Draw!")

else:
    if n_steps % 2 == alphazero_turn:
        print("Alpha-Zero Win!")
    else:
        print("You Win!")


Communication Successfully started
Sent command: 'S'
Arduino Response: Done: drawGameBoard

n_steps: 1, alphazero_turn
Sent command: O0
Arduino Response: Done: drawCircle

is_done: False
n_steps: 2, human_turn


2025-02-25 18:41:43.534 python[2028:63392] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


카메라 0가 열렸습니다.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


Falied to find 4 corners.
Falied to find 4 corners.


2025-02-25 18:41:50.519 python[2028:63392] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-25 18:41:50.520 python[2028:63392] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to find 4 corners.
Falied to fi